In [57]:
import requests
import json

# Get List of Projets and assign in variable called p_list
project_list = 'https://gdc-api.nci.nih.gov/projects?size=1000'

projects = requests.get(project_list)
data = json.loads(projects.text)
current_data = data['data']['hits']

p_list = []
for i in range(len(current_data)):
    p_list.append(current_data[i]['project_id'])
    
# Perform API Requests and append to a list titled 'Requests'
API_URL = 'https://gdc-api.nci.nih.gov/'
Endpoint = 'files'
Fields = 'file_size'
Size = '100000'

Requests = []
for Project in p_list:
    Filters = '%7B%22op%22%3A%22and%22%2C%22content%22%3A%5B%7B%22op%22%3A%22in%22%2C%22content%22%3A%7B%22field%22%3A%22cases.project.project_id%22%2C%22value%22%3A%5B%22'+Project+'%22%5D%7D%7D%5D%7D'
    Request = API_URL+Endpoint+'?'+'filters='+Filters+'&'+'fields='+Fields+'&'+'size='+Size
    Requests.append(Request)

# Extract the data from the API requests
Data = []
for RQ in Requests:
        r = requests.get(RQ)
        data = json.loads(r.text)
        current_data = data['data']['hits']
        Data.append(current_data)

# Get file count and file size and append to lists
File_Count = []
File_Size = []
for P in range(len(Data)):
    File_Count.append(len(Data[P]))
    
    File_Size_List = []
    for results in range(len(Data[P])):
        File_Size_List.append(Data[P][results]['file_size'])
    
    File_Size_List_Int = map(int, File_Size_List)
    File_Size.append(sum(File_Size_List_Int))


In [69]:
# Create pandas data frame with this information

# Convert to Gigabytes
# Size = [i / 1000000000 for i in File_Size]
# print(Size)

import pandas as pd
List = pd.DataFrame(index=p_list,data={'File Count': File_Count,'Cumulative File Size (Bytes)': File_Size})
List

,Cumulative File Size (Bytes),File Count
TCGA-LIHC,16718517073550,9511
TCGA-HNSC,17644256256749,12895
TCGA-COAD,26110619014719,11824
TCGA-PAAD,15884676588639,4433
TARGET-WT,3818225114888,1324
TCGA-UCS,2268332370353,1364
TCGA-UVM,1913462686741,1928
TCGA-STAD,19377661138371,10731
TCGA-LUSC,21583072317528,13124
TCGA-UCEC,22192513228918,13604


In [71]:
List.to_csv(path_or_buf='Files_Per_Project.tsv',sep='\t', header=True)